# Importing Libraries

In [2]:
import pandas as pd
import numpy as np
from z3 import *
from itertools import combinations
import sys
sys.path.append('../../scripts/biota/')
from BIoTA_Control import *

# Fixed Parameter

In [3]:
CO2_FRESH_AIR = 400             # CO2 concentration (ppm) of fresh air
TEMP_FRESH_AIR = 91.4           # Temperature (33 degree F) of the fresh air
CP_AIR = 1.026                  # Specific heat of fresh air
DEF_TEMP_SUPPLY_AIR =  55.4     # Default temperature (degree fahrenheit) of supply air (13 degree celsius)
MINUTES_IN_A_DAY = 1440         # Number of minutes in a day
OFF_PEAK_ENERGY_COST = 0.34     # OFF-PEAK energy cost (USD)
ON_PEAK_ENERGY_COST = 0.48      # ON-PEAK energy cost (USD)
ON_PEAK_START_SLOT = 960        # ON-PEAK start time (minute in a day)
ON_PEAK_END_SLOT = 1260         # ON-PEAK end time (minute in a day)
BATTER_STORAGE = 0.48           # Energy (kWh) produced by battery

# Variable Parameters

In [4]:
zone_temp_setpoint = [0, 75.2, 75.2, 75.2, 75.2]     # list of temperature (fahrenheit) setpoint of the different zones
zone_co2_setpoint = [0, 1000, 1000, 1000, 1000]      # list of CO2 (ppm) setpoint of the corresponding zones
control_time = 1                                     # control time (in minute)

# Accessing Zone-Activity Information

In [5]:
zones = pd.read_excel('../../data/Aras-Information.xlsx', sheet_name='Zone-Info')
zone_volume = zones["Volume (cf)"].to_list()                  # Zones' volumes (cubic feet)
pp_co2 = zones["CO2 Emission by Occupant (cfm)"].to_list()    # CO2 Emission by Occupant (cfm)
pp_heat = zones["Heat Radiation by Occupant (W)"].to_list()   # Heat Radiation by Occupant (W)
load = zones["Heat Radiated by Appliances (W)"].to_list()     # Heat radiated by Appliances (W)
        
activities = pd.read_excel('../../data/Aras-Information.xlsx', sheet_name='Activity-Info')

activity_zone_map = dict()
for i in range(len(activities)):
    activity_zone_map[int(activities["Activity ID"][i])] = int(activities["Zone ID"][i])

# Generating Possible Combination of Occupants

In [6]:
import itertools
all_samples = list(itertools.permutations([1, 1, 0, 0, 0]))
all_samples += list(itertools.permutations([2, 0, 0, 0, 0]))
# however, there are repetations

unique_samples = set()

for data in all_samples:    
    unique_samples.add(data)
    
unique_samples

{(0, 0, 0, 0, 2),
 (0, 0, 0, 1, 1),
 (0, 0, 0, 2, 0),
 (0, 0, 1, 0, 1),
 (0, 0, 1, 1, 0),
 (0, 0, 2, 0, 0),
 (0, 1, 0, 0, 1),
 (0, 1, 0, 1, 0),
 (0, 1, 1, 0, 0),
 (0, 2, 0, 0, 0),
 (1, 0, 0, 0, 1),
 (1, 0, 0, 1, 0),
 (1, 0, 1, 0, 0),
 (1, 1, 0, 0, 0),
 (2, 0, 0, 0, 0)}

# Tabulate the Control Costs for all Possible Combinations

In [7]:
# convert list to a tuple
def to_tuple(a):
    try:
        return tuple(to_tuple(i) for i in a)
    except TypeError:
        return a
    
dict_control_cost = dict()
for sample in unique_samples:
    zone_occupant = list(sample)    
    dict_control_cost[sample] = control_cost(zones, zone_occupant, zone_temp_setpoint, zone_volume, pp_co2, pp_heat, load, zone_co2_setpoint, control_time)
dict_control_cost

{(1, 0, 1, 0, 0): 0.006996611666666667,
 (1, 1, 0, 0, 0): 0.006524535000000001,
 (0, 0, 1, 0, 1): 0.036949184833333336,
 (2, 0, 0, 0, 0): 0.0,
 (0, 0, 0, 2, 0): 0.062190949333333335,
 (0, 0, 0, 0, 2): 0.03490514633333333,
 (0, 1, 0, 0, 1): 0.03647710816666667,
 (1, 0, 0, 0, 1): 0.029952573166666666,
 (0, 0, 1, 1, 0): 0.06309208633333334,
 (0, 2, 0, 0, 0): 0.012034600000000001,
 (0, 0, 0, 1, 1): 0.08604804783333334,
 (0, 1, 0, 1, 0): 0.06262000966666667,
 (0, 0, 2, 0, 0): 0.009909890000000001,
 (0, 1, 1, 0, 0): 0.013521146666666668,
 (1, 0, 0, 1, 0): 0.056095474666666666}

# Reading SHATTER Attack-Schedule Datasets

In [8]:
house_A_occ_1_final_schedule = pd.read_csv("../../data/shatter/DBSCAN_House-A_Occ-1_SHATTER_Schedule.csv").values
house_A_occ_2_final_schedule = pd.read_csv("../../data/shatter/DBSCAN_House-A_Occ-2_SHATTER_Schedule.csv").values
house_B_occ_1_final_schedule = pd.read_csv("../../data/shatter/DBSCAN_House-B_Occ-1_SHATTER_Schedule.csv").values
house_B_occ_2_final_schedule = pd.read_csv("../../data/shatter/DBSCAN_House-B_Occ-2_SHATTER_Schedule.csv").values
house_A_occ_1_greedy_schedule =  pd.read_csv("../../data/shatter/DBSCAN_House-A_Occ-1_Greedy_Schedule.csv").values
house_A_occ_2_greedy_schedule =  pd.read_csv("../../data/shatter/DBSCAN_House-A_Occ-2_Greedy_Schedule.csv").values
house_B_occ_1_greedy_schedule =  pd.read_csv("../../data/shatter/DBSCAN_House-B_Occ-1_Greedy_Schedule.csv").values
house_B_occ_2_greedy_schedule =  pd.read_csv("../../data/shatter/DBSCAN_House-B_Occ-2_Greedy_Schedule.csv").values
house_A_control_dataset = pd.read_csv('../../data/processed/Processed-Dataframe_House-A_BIoTA.csv')
house_B_control_dataset = pd.read_csv('../../data/processed/Processed-Dataframe_House-A_BIoTA.csv')

In [9]:
house_A_control_dataset

,Day,Minute,Outdoor (Zone - 0) Occupant,Bedroom (Zone - 1) Occupant,Livingroom (Zone - 2) Occupant,Kitchen (Zone - 3) Occupant,Bathroom (Zone - 4) Occupant
0,1,0,0,1,1,0,0
1,1,1,0,1,1,0,0
2,1,2,0,1,1,0,0
3,1,3,0,1,1,0,0
4,1,4,0,1,1,0,0
...,...,...,...,...,...,...,...
43195,30,1435,1,0,1,0,0
43196,30,1436,1,0,1,0,0
43197,30,1437,1,0,1,0,0
43198,30,1438,1,0,1,0,0


# SHATTER Attack Cost Calculation for House-A

In [13]:
shatter_cost_house_A = []
unit_energy_costs = []
energy_consumptions = []
prev_day = -1
battery_usage = 0 # in kWh
for j in range(0, 43200, 1440):
    print(j)
    for i in range(len(house_A_occ_1_final_schedule)):
        current_minute = i     
        control_zone_occupants = house_A_control_dataset.iloc[j + i][2:].values

        zone_occupants = [0, 0, 0, 0, 0]
        if control_zone_occupants[0] == 2:
            zone_occupants[0] = 2
        elif control_zone_occupants[0] == 1:
            zone_occupants[0] = 1
            zone_occupants[int(house_A_occ_1_final_schedule[i])] += 1
        elif control_zone_occupants[0] == 0:
            zone_occupants[int(house_A_occ_1_final_schedule[i])] += 1
            zone_occupants[int(house_A_occ_2_final_schedule[i])] += 1
        energy_consumption = dict_control_cost[to_tuple(zone_occupants)]

        if current_minute < ON_PEAK_START_SLOT or current_minute > ON_PEAK_END_SLOT:
            unit_energy_cost = OFF_PEAK_ENERGY_COST
        else:
            if battery_usage < BATTER_STORAGE:
                battery_usage += energy_consumption
                unit_energy_cost = OFF_PEAK_ENERGY_COST
            else:
                unit_energy_cost = ON_PEAK_ENERGY_COST

        current_cost = energy_consumption * unit_energy_cost
        shatter_cost_house_A.append(current_cost)
        unit_energy_costs.append(unit_energy_cost)
        energy_consumptions.append(energy_consumption)

        #shatter_cost_house_A.append(dict_control_cost[to_tuple(zone_occupants)])  
sum(shatter_cost_house_A)

0
1440
2880
4320
5760
7200
8640
10080
11520
12960
14400
15840
17280
18720
20160
21600
23040
24480
25920
27360
28800
30240
31680
33120
34560
36000
37440
38880
40320
41760


549.581020299794

# SHATTER Control Cost Calculation for House-B

In [23]:
shatter_cost_house_B = []
unit_energy_costs = []
energy_consumptions = []
prev_day = -1
battery_usage = 0 # in kWh
for j in range(0, 43200, 1440):
    print(j)
    for i in range(len(house_B_occ_1_final_schedule)):
        current_minute = i    
        zone_occupants = [0, 0, 0, 0, 0]
        control_zone_occupants = house_B_control_dataset.iloc[j + i][2:].values

        zone_occupants = [0, 0, 0, 0, 0]
        if control_zone_occupants[0] == 2:
            zone_occupants[0] = 2
        elif control_zone_occupants[0] == 1:
            zone_occupants[0] = 1
            zone_occupants[int(house_B_occ_1_final_schedule[i])] += 1
        elif control_zone_occupants[0] == 0:
            zone_occupants[int(house_B_occ_1_final_schedule[i])] += 1
            zone_occupants[int(house_B_occ_2_final_schedule[i])] += 1
        energy_consumption = dict_control_cost[to_tuple(zone_occupants)]

        if current_minute < ON_PEAK_START_SLOT or current_minute > ON_PEAK_END_SLOT:
            unit_energy_cost = OFF_PEAK_ENERGY_COST
        else:
            if battery_usage < BATTER_STORAGE:
                battery_usage += energy_consumption
                unit_energy_cost = OFF_PEAK_ENERGY_COST
            else:
                unit_energy_cost = ON_PEAK_ENERGY_COST

        current_cost = energy_consumption * unit_energy_cost
        shatter_cost_house_B.append(current_cost)
        unit_energy_costs.append(unit_energy_cost)
        energy_consumptions.append(energy_consumption)

sum(shatter_cost_house_B)

0
1440
2880
4320
5760
7200
8640
10080
11520
12960
14400
15840
17280
18720
20160
21600
23040
24480
25920
27360
28800
30240
31680
33120
34560
36000
37440
38880
40320
41760


299.688077094743

# Greedy Attack Cost Calculation for House-A

In [20]:
greedy_cost_house_A = []
unit_energy_costs = []
energy_consumptions = []
prev_day = -1
battery_usage = 0 # in kWh

for j in range(0, 43200, 1440):
    print(j)
    for i in range(len(house_A_occ_1_greedy_schedule)):
        current_minute = i    
        control_zone_occupants = house_A_control_dataset.iloc[j + i][2:].values

        zone_occupants = [0, 0, 0, 0, 0]
        if control_zone_occupants[0] == 2:
            zone_occupants[0] = 2
        elif control_zone_occupants[0] == 1:
            zone_occupants[0] = 1
            zone_occupants[int(house_A_occ_1_greedy_schedule[i])] += 1
        elif control_zone_occupants[0] == 0:
            zone_occupants[int(house_A_occ_1_greedy_schedule[i])] += 1
            zone_occupants[int(house_A_occ_2_greedy_schedule[i])] += 1

        energy_consumption = dict_control_cost[to_tuple(zone_occupants)]

        if current_minute < ON_PEAK_START_SLOT or current_minute > ON_PEAK_END_SLOT:
            unit_energy_cost = OFF_PEAK_ENERGY_COST
        else:
            if battery_usage < BATTER_STORAGE:
                battery_usage += energy_consumption
                unit_energy_cost = OFF_PEAK_ENERGY_COST
            else:
                unit_energy_cost = ON_PEAK_ENERGY_COST

        current_cost = energy_consumption * unit_energy_cost
        greedy_cost_house_A.append(current_cost)
        unit_energy_costs.append(unit_energy_cost)
        energy_consumptions.append(energy_consumption)
sum(greedy_cost_house_A) 

0
1440
2880
4320
5760
7200
8640
10080
11520
12960
14400
15840
17280
18720
20160
21600
23040
24480
25920
27360
28800
30240
31680
33120
34560
36000
37440
38880
40320
41760


517.5115231873491

# Greedy Attack Cost Calculation for House-B

In [22]:
greedy_cost_house_B = []
unit_energy_costs = []
energy_consumptions = []
prev_day = -1
battery_usage = 0 # in kWh

for j in range(0, 43200, 1440):
    print(j)
    for i in range(len(house_B_occ_1_greedy_schedule)):
        current_minute = i    
        control_zone_occupants = house_B_control_dataset.iloc[j + i][2:].values

        zone_occupants = [0, 0, 0, 0, 0]
        if control_zone_occupants[0] == 2:
            zone_occupants[0] = 2
        elif control_zone_occupants[0] == 1:
            zone_occupants[0] = 1
            zone_occupants[int(house_B_occ_1_greedy_schedule[i])] += 1
        elif control_zone_occupants[0] == 0:
            zone_occupants[int(house_B_occ_1_greedy_schedule[i])] += 1
            zone_occupants[int(house_B_occ_2_greedy_schedule[i])] += 1

        energy_consumption = dict_control_cost[to_tuple(zone_occupants)]

        if current_minute < ON_PEAK_START_SLOT or current_minute > ON_PEAK_END_SLOT:
            unit_energy_cost = OFF_PEAK_ENERGY_COST
        else:
            if battery_usage < BATTER_STORAGE:
                battery_usage += energy_consumption
                unit_energy_cost = OFF_PEAK_ENERGY_COST
            else:
                unit_energy_cost = ON_PEAK_ENERGY_COST

        current_cost = energy_consumption * unit_energy_cost
        greedy_cost_house_B.append(current_cost)
        unit_energy_costs.append(unit_energy_cost)
        energy_consumptions.append(energy_consumption)
sum(greedy_cost_house_B)

0
1440
2880
4320
5760
7200
8640
10080
11520
12960
14400
15840
17280
18720
20160
21600
23040
24480
25920
27360
28800
30240
31680
33120
34560
36000
37440
38880
40320
41760


307.060545675663

# Reading BIoTA Attack Dataset

In [12]:
biota_house_A = pd.read_csv("../../data/biota/BIoTA-Attack-Dataframe_House-A.csv")
biota_house_B = pd.read_csv("../../data/biota/BIoTA-Attack-Dataframe_House-B.csv")

# Day-wise BIoTA Attack Cost House-A

In [13]:
day_wise_biota_attack_cost_house_A = []
for i in range(0, 43200, 1440):
    day_wise_biota_attack_cost_house_A.append(sum(biota_house_A["Attack Control Cost ($)"][i:i+1400]))
total_biota_attack_cost_house_A = sum(day_wise_biota_attack_cost_house_A)
total_biota_attack_cost_house_A

775.8347225093927

# Day-wise BIoTA Attack Cost House-B

In [14]:
day_wise_biota_attack_cost_house_B = []
for i in range(0, 43200, 1440):
    day_wise_biota_attack_cost_house_B.append(sum(biota_house_B["Attack Control Cost ($)"][i:i+1400]))
total_biota_attack_cost_house_B = sum(day_wise_biota_attack_cost_house_B)
total_biota_attack_cost_house_B

518.5026439412262